# 0. 라이브러리 & 데이터

## 0.1 라이브러리

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [26]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.simplefilter("ignore")

## 0.2 데이터

In [27]:
case_fraud = pd.read_csv('../1_Data/case_fraud.csv')
case_fraud.head(1)

,index,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,사건종류코드,판결유형,선고,판례상세링크,판시사항,판결요지,참조조문,참조판례,판례내용
0,20,238021,특정경제범죄가중처벌등에관한법률위반(사기)·사기,2023도12424,2023.11.16,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=yeorii&target=prec&ID=23...,"유사수신행위를 금지·처벌하는 유사수신행위의 규제에 관한 법률 제6조 제1항, 제3...","유사수신행위의 규제에 관한 법률(이하 ‘유사수신행위법’이라 한다) 제6조 제1항, ...","유사수신행위의 규제에 관한 법률 제3조, 제6조 제1항, 형법 제37조, 제347조...",내용없음,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 장용배【배상신청인】 배...


# 1. 판례내용 전처리

## 1.1 이유 이후로 자르기

In [28]:
case_fraud['판례내용_전처리'] = case_fraud.판례내용.str.split('유】|유]').str[1]

In [29]:
case_fraud.판례내용_전처리[:3]

0      상고이유를 판단한다.  1. 불가벌적 사후행위에 대한 판단「유사수신행위의 규제에...
1      상고이유를 판단한다.   1. 피고인 1, 피고인 2, 피고인 3, 피고인 4,...
2      직권으로 판단한다.   1. 의료법 위반의 점에 관한 공소사실의 요지   피고인...
Name: 판례내용_전처리, dtype: object

## 1.2 한자 처리

https://github.com/suminb/hanja

In [30]:
import hanja
from hanja import hangul

In [31]:
hanja.is_hanja('潘')

True

In [32]:
def hanja_to_hangul(sent):
    return hanja.translate(sent, 'substitution')

In [33]:
case_fraud.판례내용_전처리 = case_fraud.판례내용_전처리.apply(hanja_to_hangul)

## 1.3 한글 외의 특수 문자 제거

In [34]:
import re

def remove_special_char(sent):
    return re.sub('[^가-힣 ]', '', sent)

In [35]:
case_fraud.판례내용_전처리 = case_fraud.판례내용_전처리.apply(remove_special_char)
case_fraud.판례내용_전처리[:3]

0      상고이유를 판단한다   불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 ...
1      상고이유를 판단한다    피고인  피고인  피고인  피고인  피고인 에 대한 의...
2      직권으로 판단한다    의료법 위반의 점에 관한 공소사실의 요지   피고인은 의...
Name: 판례내용_전처리, dtype: object

In [36]:
case_fraud['판례내용_전처리'][0]

'  상고이유를 판단한다   불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다 제조 제항 제조를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법 제조 제항의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위법 위반죄의 불가벌적 사후행위가 되는 것이 아니라 별죄인 사기죄를 구성한다  원심은 판시와 같은 이유로 피고인의   경 피해자 공소외인에 대한 판시 사기의 점에 관한 공소사실에 대하여 유사수신행위법 제조에서 금지하는 유사수신행위에는 기망행위가 포함되어 있지 않고 유사수신행위법 위반죄와 사기죄는 구성요건을 달리하는 별개의 범죄로서 서로 행위의 태양이나 보호법익을 달리한다는 등의 이유로 기존 범죄인 유사수신행위법 위반죄의 가벌적 평가범위 내에 흡수되는 불가벌적 사후행위에 해당하지 않는다고 판단하였다  원심판결 이유를 위 법리 및 기록에 비추어 살펴보면 이 부분 원심판단에 일부 적절하지 않은 점이 있으나 피고인의   경 피해자 공소외인에 대한 판시 사기죄가 유사수신행위법 위반죄의 불가벌적 사후행위에 해당하지 않는다고 본 원심의 판단에 불가벌적 사후행위에 관한 법리를 오해함으로써 판결에 영향을 미친 잘못이 없다   나머지 상고이유에 대한 판단  이 부분 상고이유는 형사소송법 제조 제호에서 정한 형보다 가벼운 형이 선고된 이 사건에서 사실오인법리오해를 내세우며 실질적으로 원심의 증거 선택 및 증명력에 관한 판단 또는 이에 기초한 사실인정을 탓하는 것이거나 원심이 인정한 사실과 다른 사실관계를 전제로 법리오해를 지적하는 취지의 주장 또는 형이 너무 무거워 부당하다는 취지의 주장에 해당하여 모두 적법한 상고이유로 볼 수 없다   결론  그러므로

## [Skip] 띄어쓰기 처리

In [37]:
# from pykospacing import Spacing

# spacing = Spacing() 

In [38]:
# spacing(case_fraud['판례내용_전처리'][0])

## 1.4 토크나이저

### 1.4.1 토크나이저 비교

#### [Skip] SoyNLP

In [39]:
# import urllib.request
# from soynlp import DoublespaceLineCorpus
# from soynlp.word import WordExtractor

In [40]:
# # 훈련 데이터를 다수의 문서로 분리
# # urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")
# corpus = DoublespaceLineCorpus("2016-10-20.txt")
# len(corpus)

In [41]:
# # 학습하기
# word_extractor = WordExtractor()
# word_extractor.train(corpus)
# word_score_table = word_extractor.extract()
# scores = {word:score.cohesion_forward for word, score in word_score_table.items()}

In [42]:
# from soynlp.tokenizer import MaxScoreTokenizer

# maxscore_tokenizer = MaxScoreTokenizer(scores=scores)

#### KoNLPy

In [43]:
import konlpy
konlpy.jvm.init_jvm(jvmpath=None, max_heap_size=2048)

In [44]:
from konlpy.tag import *

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

#### 비교

In [45]:
sent = case_fraud.판례내용_전처리[0][:300]

print('Input:', sent, end='\n\n')

#print('Hannanum:', hannanum.morphs(sent), end='\n\n')
print(kkma.morphs(sent), end='\n\n')
print(komoran.morphs(sent), end='\n\n')
# print(okt.morphs(sent), end='\n\n')
#print('SoyNLP:', maxscore_tokenizer.tokenize(sent))

Input:   상고이유를 판단한다   불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다 제조 제항 제조를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법 제조 제항의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위

['상고', '이유', '를', '판단', '하', 'ㄴ다', '불가', '벌', 'ㄹ', '적', '사후', '행위', '에', '대하', 'ㄴ', '판단', '유사', '수신', '행위', '의', '규제', '에', '관하', 'ㄴ', '법률', '이하', '유사', '수신', '행위법', '이', '라', '하', 'ㄴ다', '제조', '제항', '제조', '를', '위반', '하', 'ㄴ', '행위', '는', '그', '자체', '가', '사기', '행위', '에', '해당', '하', 'ㄴ다', '거나', '사기', '행위', '를', '반드시', '포함', '하', 'ㄴ다고', '하', 'ㄹ', '수', '없', '고', '유사', '수신', '행위법', '위반죄', '가', '형법', '제조', '제항', '의', '사기죄', '와', '구성', '요건', '을', '달리하', '는', '별개', '의', '범죄', '로서', '서로', '보호', '법익', '이', '다른', '이상', '유사', '수신', '행위', '를', '하', 'ㄴ', '자가', '출자자', '에게', '별도', '의', '기망', '행위', '를', '하여', '유사', '수신', '행위', '로', '조달', '받', '은', '자금', '의', '전부', '또는', '일부', '를', '다시', '투자', '받', '는

In [46]:
sent = case_fraud.판례내용_전처리[0][:300]

print('Input:', sent, end='\n\n')

#print('Hannanum:', hannanum.morphs(sent), end='\n\n')
print(kkma.nouns(sent), end='\n\n')
print(komoran.nouns(sent), end='\n\n')
# print(okt.morphs(sent), end='\n\n')
#print('SoyNLP:', maxscore_tokenizer.tokenize(sent))

Input:   상고이유를 판단한다   불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다 제조 제항 제조를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법 제조 제항의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위

['상고', '상고이유', '이유', '판단', '불가', '적', '사후', '사후행위', '행위', '판단유사수신행위', '유사', '수신', '규제', '법률', '법률이하', '이하', '유사수신행위법', '행위법', '제조', '제항', '위반', '자체', '사기', '사기행위', '해당', '포함', '수', '위반죄', '형법', '사기죄', '구성', '구성요건', '요건', '별개', '범죄', '서로', '보호', '보호법익', '법익', '이상', '유사수신행위', '자가', '출자자', '별도', '기망', '기망행위', '조달', '자금', '전부', '일부', '투자', '새', '침해']

['상고', '이유', '판단', '불가벌적 사후행위', '판단', '유사', '수신', '행위', '규제', '법률', '이하', '유사', '수신', '행위', '법', '제조', '항', '제조', '위반', '행위', '자체', '사기', '행위', '해당', '사기', '행위', '포함', '수', '유사', '수신', '행위', '법', '위반', '죄', '형법', '제조', '항의', '사기죄', '구성요건', '별개', '범죄', '보호법익', '이상', '유사', '수신', '행위', '자', '출자', '자', '별도', '기망행위', '유사', '수신', '행위', '조달', '자금

### 1.4.2 토큰화 전 문장 내 단어 개수 체크

In [47]:
case_fraud['판례내용_전처리_글자수'] = case_fraud['판례내용_전처리'].apply(lambda x: len(x))
case_fraud.판례내용_전처리_글자수.value_counts().sort_index() # 45자부터 467481자까지

45        1
154       1
156       1
159       1
203       1
227       1
242       1
271       1
277       1
293       1
307       2
318       1
319       1
320       2
324       1
325       1
329       1
332       1
339       2
345       1
346       1
347       1
349       1
357       1
363       1
369       1
372       1
375       1
380       1
397       1
399       2
401       1
405       2
406       1
409       1
413       1
417       1
420       1
421       1
422       1
423       1
426       1
427       1
430       1
432       1
434       1
438       1
447       1
453       1
455       1
460       1
464       1
468       2
471       2
474       1
478       1
482       1
483       1
485       1
486       1
487       2
490       1
491       1
493       2
498       1
503       1
506       1
508       1
511       1
513       1
516       1
518       1
519       1
521       1
523       1
524       1
527       2
531       1
537       1
540       1
542       1
545       1
546       1
548 

In [48]:
print((case_fraud['판례내용_전처리_글자수'] > 90000).sum()) # 9만 자 넘는 판례 15개 있음
case_fraud = case_fraud[case_fraud['판례내용_전처리_글자수'] < 90000] # 9만 자 넘는 판례 15개 제거
print((case_fraud['판례내용_전처리_글자수'] > 90000).sum())

15
0


### 1.4.3 토큰화 & 의미 있는 명사만 추출

In [60]:
# kkmaTag = []

# for sent in tqdm(case_fraud.판례내용_전처리):
#     kkmaTagList = kkma.pos(sent)
#     for (text, pos) in kkmaTagList:
#         if pos == 'NNG':
#            kkmaTag.append(text)

# with open('./vocab_list/kkma.txt', 'w+') as file:
#     file.write('\n'.join(kkmaTag))

100%|██████████| 2253/2253 [39:09<00:00,  1.04s/it]  


['상고',
 '이유',
 '판단',
 '불가',
 '사후',
 '행위',
 '판단',
 '유사',
 '수신',
 '행위',
 '규제',
 '법률',
 '이하',
 '유사',
 '수신',
 '행위법',
 '제조',
 '제항',
 '제조',
 '위반',
 '행위',
 '자체',
 '사기',
 '행위',
 '해당',
 '사기',
 '행위',
 '포함',
 '유사',
 '수신',
 '행위법',
 '위반죄',
 '형법',
 '제조',
 '제항',
 '사기죄',
 '구성',
 '요건',
 '별개',
 '범죄',
 '서로',
 '보호',
 '법익',
 '이상',
 '유사',
 '수신',
 '행위',
 '자가',
 '출자자',
 '별도',
 '기망',
 '행위',
 '유사',
 '수신',
 '행위',
 '조달',
 '자금',
 '전부',
 '일부',
 '투자',
 '행위',
 '유사',
 '수신',
 '행위법',
 '위반죄',
 '새',
 '보호',
 '법익',
 '침해',
 '유사',
 '수신',
 '행위법',
 '위반죄',
 '불가',
 '사후',
 '행위',
 '별',
 '죄인',
 '사기죄',
 '구성',
 '원심',
 '판시',
 '이유',
 '피고',
 '인의',
 '경',
 '피해자',
 '공소',
 '외인',
 '판시',
 '사기',
 '점',
 '공소',
 '사실',
 '유사',
 '수신',
 '행위법',
 '제조',
 '금지',
 '유사',
 '수신',
 '행위',
 '기망',
 '행위',
 '포함',
 '유사',
 '수신',
 '행위법',
 '위반죄',
 '사기죄',
 '구성',
 '요건',
 '별개',
 '범죄',
 '서로',
 '행위',
 '태양',
 '보호',
 '법익',
 '이유',
 '기존',
 '범죄인',
 '유사',
 '수신',
 '행위법',
 '위반죄',
 '가벌',
 '평가',
 '범위',
 '흡수',
 '불가',
 '사후',
 '행위',
 '해당',
 '판단',
 '원심',
 '판결',
 '이유',
 '위',
 '법리',
 '기록',


In [61]:
# komoranTag = []

# for sent in tqdm(case_fraud.판례내용_전처리):
#     komoranTagList = komoran.pos(sent)
#     for (text, pos) in komoranTagList:
#         if pos in ['NNG', 'NNP']:
#            komoranTag.append(text)

# with open('./vocab_list/komoran.txt', 'w+') as file:
#     file.write('\n'.join(komoranTag))

100%|██████████| 2253/2253 [11:27<00:00,  3.28it/s] 


['상고',
 '이유',
 '판단',
 '불가벌적 사후행위',
 '판단',
 '유사',
 '수신',
 '행위',
 '규제',
 '법률',
 '이하',
 '유사',
 '수신',
 '행위',
 '제조',
 '항',
 '제조',
 '위반',
 '행위',
 '자체',
 '사기',
 '행위',
 '해당',
 '사기',
 '행위',
 '포함',
 '유사',
 '수신',
 '행위',
 '위반',
 '죄',
 '형법',
 '제조',
 '항의',
 '사기죄',
 '구성요건',
 '별개',
 '범죄',
 '보호법익',
 '이상',
 '유사',
 '수신',
 '행위',
 '출자',
 '별도',
 '기망행위',
 '유사',
 '수신',
 '행위',
 '조달',
 '자금',
 '일부',
 '투자',
 '행위',
 '유사',
 '수신',
 '행위',
 '위반',
 '죄',
 '보호법익',
 '침해',
 '유사',
 '수신',
 '행위',
 '위반',
 '죄',
 '불가벌적 사후행위',
 '별',
 '죄인',
 '사기죄',
 '구성',
 '원심',
 '판시',
 '이유',
 '피고인',
 '경',
 '피해자',
 '공소',
 '외인',
 '판시',
 '사기',
 '공소사실',
 '유사',
 '수신',
 '행위',
 '제조',
 '금지',
 '유사',
 '수신',
 '행위',
 '기망행위',
 '포함',
 '유사',
 '수신',
 '행위',
 '위반',
 '죄',
 '사기죄',
 '구성요건',
 '별개',
 '범죄',
 '행위',
 '태양',
 '보호법익',
 '이유',
 '기존',
 '범죄인',
 '유사',
 '수신',
 '행위',
 '위반',
 '죄',
 '가벌',
 '평가',
 '범위',
 '흡수',
 '불가벌적 사후행위',
 '해당',
 '판단',
 '원심',
 '판결',
 '이유',
 '위',
 '법리',
 '기록',
 '부분',
 '원심',
 '판단',
 '일부',
 '피고인',
 '경',
 '피해자',
 '공소',
 '외인',
 '판시',
 '사기죄',
 '유사',
 '수신'

In [84]:
kkmaVocab = []
with open("./vocab_list/kkma.txt", "r") as file:
    for i in file:
        kkmaVocab.append(i.strip())
kkmaVocab[:10]

['상고', '이유', '판단', '불가', '사후', '행위', '판단', '유사', '수신', '행위']

In [85]:
komoranVocab = []
with open("./vocab_list/kkma.txt", "r") as file:
    for i in file:
        komoranVocab.append(i.strip())
komoranVocab[:10]

['상고', '이유', '판단', '불가', '사후', '행위', '판단', '유사', '수신', '행위']

## 1.5 불용어

### 1.5.1 불용어 제거 위한 단어 리스트 & 카운트 확인

In [86]:
from collections import Counter

In [87]:
Counter(kkmaVocab).most_common()

[('피고인', 87957),
 ('공소', 48524),
 ('위', 37589),
 ('사실', 30908),
 ('원심', 24021),
 ('사건', 19878),
 ('판결', 19198),
 ('공', 18361),
 ('회사', 18265),
 ('제조', 18202),
 ('소외', 17938),
 ('점', 16450),
 ('이유', 15916),
 ('피해자', 14788),
 ('인정', 14274),
 ('의', 12757),
 ('범죄', 12643),
 ('행위', 12222),
 ('판단', 12137),
 ('부분', 11648),
 ('경', 10944),
 ('증거', 10520),
 ('진술', 10305),
 ('제항', 9947),
 ('기재', 9792),
 ('주장', 8984),
 ('지급', 8575),
 ('상고', 8316),
 ('선고', 8309),
 ('주식회사', 7956),
 ('경우', 7596),
 ('원', 7300),
 ('이', 7176),
 ('외', 7160),
 ('기록', 7069),
 ('은', 6526),
 ('계약', 6455),
 ('명의', 6317),
 ('후', 6004),
 ('위반', 5989),
 ('판시', 5966),
 ('형법', 5820),
 ('작성', 5754),
 ('범행', 5746),
 ('해당', 5729),
 ('사기', 5725),
 ('법률', 5657),
 ('자금', 5648),
 ('심', 5597),
 ('법리', 5593),
 ('내용', 5385),
 ('관계', 5200),
 ('검사', 5167),
 ('생략', 5120),
 ('로', 4961),
 ('제호', 4933),
 ('관련', 4932),
 ('사정', 4816),
 ('의사', 4766),
 ('위법', 4741),
 ('대금', 4694),
 ('편취', 4650),
 ('사용', 4517),
 ('사기죄', 4509),
 ('취지', 4464),
 ('거래', 4

In [88]:
Counter(komoranVocab).most_common()

[('피고인', 87957),
 ('공소', 48524),
 ('위', 37589),
 ('사실', 30908),
 ('원심', 24021),
 ('사건', 19878),
 ('판결', 19198),
 ('공', 18361),
 ('회사', 18265),
 ('제조', 18202),
 ('소외', 17938),
 ('점', 16450),
 ('이유', 15916),
 ('피해자', 14788),
 ('인정', 14274),
 ('의', 12757),
 ('범죄', 12643),
 ('행위', 12222),
 ('판단', 12137),
 ('부분', 11648),
 ('경', 10944),
 ('증거', 10520),
 ('진술', 10305),
 ('제항', 9947),
 ('기재', 9792),
 ('주장', 8984),
 ('지급', 8575),
 ('상고', 8316),
 ('선고', 8309),
 ('주식회사', 7956),
 ('경우', 7596),
 ('원', 7300),
 ('이', 7176),
 ('외', 7160),
 ('기록', 7069),
 ('은', 6526),
 ('계약', 6455),
 ('명의', 6317),
 ('후', 6004),
 ('위반', 5989),
 ('판시', 5966),
 ('형법', 5820),
 ('작성', 5754),
 ('범행', 5746),
 ('해당', 5729),
 ('사기', 5725),
 ('법률', 5657),
 ('자금', 5648),
 ('심', 5597),
 ('법리', 5593),
 ('내용', 5385),
 ('관계', 5200),
 ('검사', 5167),
 ('생략', 5120),
 ('로', 4961),
 ('제호', 4933),
 ('관련', 4932),
 ('사정', 4816),
 ('의사', 4766),
 ('위법', 4741),
 ('대금', 4694),
 ('편취', 4650),
 ('사용', 4517),
 ('사기죄', 4509),
 ('취지', 4464),
 ('거래', 4

In [97]:
Counter(kkmaVocab).most_common()

[('피고인', 87957),
 ('공소', 48524),
 ('위', 37589),
 ('사실', 30908),
 ('원심', 24021),
 ('사건', 19878),
 ('판결', 19198),
 ('공', 18361),
 ('회사', 18265),
 ('제조', 18202),
 ('소외', 17938),
 ('점', 16450),
 ('이유', 15916),
 ('피해자', 14788),
 ('인정', 14274),
 ('의', 12757),
 ('범죄', 12643),
 ('행위', 12222),
 ('판단', 12137),
 ('부분', 11648),
 ('경', 10944),
 ('증거', 10520),
 ('진술', 10305),
 ('제항', 9947),
 ('기재', 9792),
 ('주장', 8984),
 ('지급', 8575),
 ('상고', 8316),
 ('선고', 8309),
 ('주식회사', 7956),
 ('경우', 7596),
 ('원', 7300),
 ('이', 7176),
 ('외', 7160),
 ('기록', 7069),
 ('은', 6526),
 ('계약', 6455),
 ('명의', 6317),
 ('후', 6004),
 ('위반', 5989),
 ('판시', 5966),
 ('형법', 5820),
 ('작성', 5754),
 ('범행', 5746),
 ('해당', 5729),
 ('사기', 5725),
 ('법률', 5657),
 ('자금', 5648),
 ('심', 5597),
 ('법리', 5593),
 ('내용', 5385),
 ('관계', 5200),
 ('검사', 5167),
 ('생략', 5120),
 ('로', 4961),
 ('제호', 4933),
 ('관련', 4932),
 ('사정', 4816),
 ('의사', 4766),
 ('위법', 4741),
 ('대금', 4694),
 ('편취', 4650),
 ('사용', 4517),
 ('사기죄', 4509),
 ('취지', 4464),
 ('거래', 4

### 1.5.2 불용어 제거

In [78]:
stopwords = []
with open("./vocab_list/stopwords.txt", "r") as file:
    for i in file:
        stopwords.append(i.strip())
stopwords

['위', '공', '점', '의', '제항', '김', '제', '가', '나', '다', '라']

In [80]:
kkmaVocab = [word for word in kkmaVocab if not word in stopwords]
kkmaVocab[:10]

['상고', '이유', '판단', '불가', '사후', '행위', '판단', '유사', '수신', '행위']

In [81]:
komoranVocab = [word for word in komoranVocab if not word in stopwords]
komoranVocab[:10]

['상고', '이유', '판단', '불가벌적 사후행위', '판단', '유사', '수신', '행위', '규제', '법률']